# Load Testing Data

In [1]:
import pandas as pd
import numpy as np
import os
import pysam

In [2]:
def parse_record(record):
    """line parser to build dataframe, supports missing tags in test data"""
    data = {
        'qname': record.query_name,
        'flag': record.flag,
        'reference': record.reference_id,
        'position': record.pos,
        'mapq': record.query_qualities,
        'cigar': record.cigarstring,
        'rnext': record.rnext, 
        'pnext': record.pnext,
        'tlen': record.tlen, 
        'sequence': record.seq,
        'quality': record.qual,
    }
    for name, tag in record.get_tags():
        data[name] = tag
    return pd.Series(data)

input_sam_file = 'data/small-gene-sorted.bam'
with pysam.AlignmentFile(input_sam_file, 'rb') as f:
    records = []
    for record in f:
        records.append(parse_record(record))

data = pd.concat(records, axis=1).T
results_scalar = {}

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:24: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


# Build Expectations for Testing Data

## Number of Reads

In [3]:
results_scalar['n_reads'] = len(data)
print(results_scalar['n_reads'])

300


## Number of Genes

In [4]:
results_scalar['n_genes'] = len(data.groupby(['GE']))
print(results_scalar['n_genes'])

8


Gene table should have 8 entries plus a header for a total of 9 lines

## Number of Molecules

Molecules are defined as a unique triplet of CB, UB, and GE

In [5]:
results_scalar['n_molecules'] = len(data.groupby(['CB', 'UB', 'GE']))
print(results_scalar['n_molecules'])

88


## Number of Fragments

Fragments are defined as molecules are (CB, UB, GE) but must additionally have a unique position

In [6]:
results_scalar['n_fragments'] = len(data.groupby(['CB', 'UB', 'GE', 'position']))
print(results_scalar['n_fragments'])

217


## Most Abundant Gene

Based on the above, at least one of the genes has to be observed more than once. Which is it? 

In [7]:
results_scalar['most_abundant'] = data.groupby(['GE']).size().idxmax().split(':')[-1]
results_scalar['most_abundant_gene_n_observations'] = data.groupby(['GE']).size().max()
print(results_scalar['most_abundant'], results_scalar['most_abundant_gene_n_observations'])

AL627309.7 245


In [8]:
results_scalar['perfect_molecule_barcodes'] = 0
for c, r in zip(data['UB'], data['UR']):
    if c.split(':')[-1] == r.split(':')[-1]:
        results_scalar['perfect_molecule_barcodes'] += 1

Calculate the alignment metrics

In [9]:
results_scalar['reads_mapped_exonic'] = sum(data['XF'] == 'XF:Z:CODING')

In [10]:
results_scalar['reads_mapped_intronic'] = sum(data['XF'] == 'XF:Z:INTRONIC')

In [11]:
results_scalar['reads_mapped_utr'] = sum(data['XF'] == 'XF:Z:UTR')

In [12]:
results_scalar['reads_mapped_uniquely'] = sum(data['NH'] == 'NH:i:1')

In [13]:
results_scalar['duplicate_reads'] = sum((data['flag'] & 1024).astype(bool))

In [14]:
results_scalar['spliced_reads'] = sum(1 for v in data['cigar'] if 'N' in v)

Calculate the higher-order metrics

In [15]:
calc_func_fraction = lambda x: sum(1 for c in x.split(':')[-1] if ord(c) > 63) / len(x.split(':')[-1])
calc_func_mean = lambda x: np.mean([ord(c) - 33 for c in x.split(':')[-1]])

data['num_UY_qual_fraction'] = data['UY'].apply(calc_func_fraction)

data['num_base_qual_fraction'] = data['quality'].apply(calc_func_fraction)
data['num_base_qual_mean'] = data['quality'].apply(calc_func_mean)

grouped_by_gene = data.groupby(['GE'])

In [16]:
results_series = {}

In [17]:
# vector values
# I changed these to retain the index to make merging into a dataframe easier, and guarantee same order. 
results_series['molecule_barcode_fraction_bases_above_30_mean'] = grouped_by_gene.mean()['num_UY_qual_fraction']
results_series['molecule_barcode_fraction_bases_above_30_variance'] = grouped_by_gene.var()['num_UY_qual_fraction']

results_series['genomic_reads_fraction_bases_quality_above_30_mean'] = grouped_by_gene.mean()['num_base_qual_fraction']
results_series['genomic_reads_fraction_bases_quality_above_30_variance'] = grouped_by_gene.var()['num_base_qual_fraction']
results_series['genomic_read_quality_mean'] = grouped_by_gene.mean()['num_base_qual_mean']
results_series['genomic_read_quality_variance'] = grouped_by_gene.var()['num_base_qual_mean']

reads_per_gene = data.groupby(['GE']).size()

In [18]:
molecules_per_gene = grouped_by_gene.apply(lambda x: len(x.groupby(['UB', 'CB']).size()))
fragments_per_gene = grouped_by_gene.apply(lambda x: len(x.groupby(['UB', 'CB', 'position']).size()))
reads_per_molecule = reads_per_gene / molecules_per_gene
reads_per_fragment = reads_per_gene / fragments_per_gene
fragments_per_molecule = fragments_per_gene / molecules_per_gene
results_series['reads_per_molecule'] = reads_per_molecule
results_series['reads_per_fragment'] = reads_per_fragment
results_series['fragments_per_molecule'] = fragments_per_molecule

# scalar values
results_scalar['fragments_with_single_read_evidence'] = np.sum(data.groupby(['CB', 'UB', 'GE', 'position']).size() == 1)
results_scalar['molecules_with_single_read_evidence'] = np.sum(data.groupby(['CB', 'UB', 'GE']).size() == 1)

In [19]:
pd.DataFrame(results_series)

,fragments_per_molecule,genomic_read_quality_mean,genomic_read_quality_variance,genomic_reads_fraction_bases_quality_above_30_mean,genomic_reads_fraction_bases_quality_above_30_variance,molecule_barcode_fraction_bases_above_30_mean,molecule_barcode_fraction_bases_above_30_variance,reads_per_fragment,reads_per_molecule
GE,,,,,,,,,
ACAP3,1.000000,36.214286,NaN,0.887755,NaN,1.000000,NaN,1.000000,1.000000
AGRN,1.000000,24.846939,NaN,0.397959,NaN,1.000000,NaN,1.000000,1.000000
AL627309.1,1.000000,25.306122,NaN,0.418367,NaN,0.800000,NaN,1.000000,1.000000
AL627309.5,1.875000,35.382261,18.336345,0.815542,0.027979,0.988462,0.001062,1.733333,3.250000
AL627309.7,2.983051,34.088130,21.543840,0.768055,0.034475,0.983265,0.005088,1.392045,4.152542
AL645608.2,1.250000,33.036443,33.657186,0.721574,0.053718,0.985714,0.001429,1.400000,1.750000
AL645608.3,1.000000,20.857143,NaN,0.163265,NaN,0.700000,NaN,1.000000,1.000000
AL645608.4,1.307692,27.269274,54.069164,0.507937,0.085290,0.944444,0.012026,1.058824,1.384615


# Write Results to File for Automated Testing

In [20]:
!pwd

/Users/carra1/projects/humancellatlas/sctools/src/sctools/test


In [21]:
print(input_sam_file.replace('.bam', ''))

data/small-gene-sorted


In [22]:
pd.Series(results_scalar).to_csv('%s_testing_knowledge_scalar.csv' % input_sam_file.replace('.bam', ''))
pd.DataFrame(results_series).to_csv('%s_testing_knowledge_series.csv' % input_sam_file.replace('.bam', ''))

In [23]:
# do a comparison of the whole 2d dataframe at once
np.allclose(
    pd.DataFrame(results_series).fillna(0).values,  # fill nans with zero, call values to get the numpy array the dataframe is based on
    pd.read_csv('%s_testing_knowledge_series.csv' % input_sam_file.replace('.bam', ''), index_col=0, header=0).fillna(0).values
)

True

In [24]:
# to get most_abundant alone: 

In [25]:
test_read_scalar = pd.read_csv('%s_testing_knowledge_scalar.csv' % input_sam_file.replace('.bam', ''), index_col=0, header=None, squeeze=True)

# extract this, we're going to drop it from the array to do some conversion to numeric
most_abundant = test_read_scalar['most_abundant'] 

# drop most abundant, convert to float, fill any NaN values with 0, and call .values to get the numpy array pandas objects are based on.
for_comparison = test_read_scalar.drop('most_abundant').astype(float).fillna(0).values


# note, have to drop the string value and convert to float before this works. 
np.allclose(
    pd.Series(results_scalar).drop('most_abundant').fillna(0).values,  # do the same thing as above to the one in memory
    for_comparison
)

True

In [26]:
# get a metric from a dataframe: 
df = pd.DataFrame(results_series)
df['genomic_read_quality_mean']

GE
ACAP3         36.214286
AGRN          24.846939
AL627309.1    25.306122
AL627309.5    35.382261
AL627309.7    34.088130
AL645608.2    33.036443
AL645608.3    20.857143
AL645608.4    27.269274
Name: genomic_read_quality_mean, dtype: float64

In [27]:
# get a numpy array from the dataframe
compare_me = df['genomic_read_quality_mean'].values

In [28]:
# compare two numpy arrays that are slightly different
eps = np.random.rand(len(compare_me)) * 1e-8
np.allclose(compare_me, compare_me + eps)


True

In [29]:
# it is actually discriminative, though
np.allclose(compare_me, np.arange(len(compare_me)))

False